## 导入数据

In [10]:
import re
import pandas as pd
import json
from datetime import datetime
import numpy as np

# 读取 SQL 文件
with open('./data/meal_list.sql', 'r', encoding='utf-8') as f:
    sql_content = f.read()

# 提取插入语句中的数据
insert_statements = re.findall(r"INSERT INTO `meal_list` VALUES \((.*?)\);", sql_content, re.S)

# 解析数据
meal_data = []
for statement in insert_statements:
    rows = statement.split("),(")
    for row in rows:
        row_data = row.split(',')
        meal_data.append([int(row_data[0]), row_data[1].strip("'"), row_data[2].strip("'")])

# 创建 DataFrame
meal_df = pd.DataFrame(meal_data, columns=['mealno', 'mealID', 'meal_name'])

# 读取 JSON 文件
with open('./data/MealRatings_201705_201706.json', 'r', encoding='utf-8') as f:
    ratings_data = json.load(f)

# 创建 DataFrame
ratings_df = pd.DataFrame(ratings_data)

# 数据预处理
# 1. 原始数据的探索分析
total_records = len(ratings_df)
total_users = ratings_df['UserID'].nunique()
total_meals = ratings_df['MealID'].nunique()
max_rating = ratings_df['Rating'].max()
min_rating = ratings_df['Rating'].min()

print(f"总纪录数: {total_records}, 总用户数: {total_users}, 总菜品数: {total_meals}, 最高评分: {max_rating}, 最低评分: {min_rating}")

# 评分分布统计
rating_distribution = ratings_df['Rating'].value_counts(normalize=True)
print("评分分布:")
print(rating_distribution)

# 检查重复评分记录
duplicated_ratings = ratings_df[ratings_df.duplicated(subset=['UserID', 'MealID'], keep=False)]
num_duplicated = len(duplicated_ratings)
print(f"重复评分记录总数: {num_duplicated}")

# 处理重复评分记录
ratings_df.sort_values(by=['UserID', 'MealID', 'ReviewTime'], ascending=[True, True, False], inplace=True)
ratings_df.drop_duplicates(subset=['UserID', 'MealID'], keep='first', inplace=True)

# 更新后的总记录数
total_records_after_dedup = len(ratings_df)
print(f"去重后的记录总数: {total_records_after_dedup}")

# 数据变换
# 编码用户和菜品
user_id_map = {user_id: idx for idx, user_id in enumerate(ratings_df['UserID'].unique())}
meal_id_map = {meal_id: idx for idx, meal_id in enumerate(ratings_df['MealID'].unique())}

ratings_df['user_code'] = ratings_df['UserID'].map(user_id_map)
ratings_df['meal_code'] = ratings_df['MealID'].map(meal_id_map)

# 数据集分割
train_ratio = 0.8
validation_ratio = 0.1
test_ratio = 0.1

train_df, validation_df, test_df = np.split(ratings_df.sample(frac=1, random_state=42), 
                                            [int(train_ratio*len(ratings_df)), 
                                             int((train_ratio+validation_ratio)*len(ratings_df))])

print(f"训练集大小: {len(train_df)}, 验证集大小: {len(validation_df)}, 测试集大小: {len(test_df)}")

# 将预处理后的数据保存到文件
train_df.to_csv('train_ratings.csv', index=False)
validation_df.to_csv('validation_ratings.csv', index=False)
test_df.to_csv('test_ratings.csv', index=False)

meal_df.to_csv('meal_list_processed.csv', index=False)

print("数据预处理完成，预处理后的数据已保存到文件。")


总纪录数: 38384, 总用户数: 5130, 总菜品数: 1685, 最高评分: 5.0, 最低评分: 1.0
评分分布:
Rating
5.0    0.547754
4.0    0.238172
3.0    0.116585
2.0    0.051063
1.0    0.046426
Name: proportion, dtype: float64
重复评分记录总数: 2518
去重后的记录总数: 37125
训练集大小: 29700, 验证集大小: 3712, 测试集大小: 3713
数据预处理完成，预处理后的数据已保存到文件。


d:\ProgramData\Anaconda3\envs\ML\lib\site-packages\numpy\core\fromnumeric.py:57: FutureWarning: 'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.
  return bound(*args, **kwds)


## 1. 基于用户的协同过滤算法

In [11]:
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

# 加载数据
ratings_df = pd.read_csv('train_ratings.csv')

# 创建用户-物品矩阵
user_item_matrix = ratings_df.pivot_table(index='user_code', columns='meal_code', values='Rating')

# 计算用户之间的相似度
user_similarity = cosine_similarity(user_item_matrix.fillna(0))


# 获取某个用户的评分
def get_user_ratings(user_id):
    return user_item_matrix.loc[user_id].dropna()


# 基于用户的协同过滤推荐
def user_based_recommendation(user_id, top_k=4, n_neigh=74):
    # 检查用户是否在用户-物品矩阵中
    if user_id not in user_item_matrix.index:
        return pd.Series()

    # 获取当前用户与其他用户的相似度分数
    user_sim_scores = pd.Series(user_similarity[user_item_matrix.index.get_loc(user_id)], index=user_item_matrix.index)
    # 删除当前用户自身的相似度分数并按相似度降序排序
    user_sim_scores = user_sim_scores.drop(user_id, errors='ignore').sort_values(ascending=False)
    # 选取前n_neigh个最相似的用户
    similar_users = user_sim_scores.index[:n_neigh]

    # 初始化推荐列表
    recommendations = pd.Series(dtype=np.float64)
    for idx, similar_user in enumerate(similar_users):
        # 获取相似用户的评分
        similar_user_ratings = get_user_ratings(similar_user)
        # 根据相似用户的评分和相似度分数计算推荐得分
        recommendations = pd.concat([recommendations, similar_user_ratings * user_sim_scores.iloc[idx]])

    # 按照推荐得分对物品进行分组和求和
    recommendations = recommendations.groupby(recommendations.index).sum()
    # 获取当前用户已评分的物品
    user_rated_items = get_user_ratings(user_id).index
    # 从推荐列表中删除当前用户已评分的物品
    recommendations = recommendations.drop(user_rated_items, errors='ignore')

    # 返回得分最高的top_k个推荐物品
    return recommendations.sort_values(ascending=False).head(top_k)


# 示例推荐
print(user_based_recommendation(0))

190    20.784436
89      7.491533
442     7.444267
212     6.120793
dtype: float64


## 2. 基于物品的协同过滤算法

In [8]:
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

# 加载数据
ratings_df = pd.read_csv('train_ratings.csv')

# 创建用户-物品矩阵
user_item_matrix = ratings_df.pivot_table(index='user_code', columns='meal_code', values='Rating')

# 创建物品-用户矩阵（转置用户-物品矩阵）
item_user_matrix = user_item_matrix.T

# 计算物品之间的相似度
item_similarity = cosine_similarity(item_user_matrix.fillna(0))


# 获取某个物品的评分
def get_item_ratings(item_id):
    return item_user_matrix.loc[item_id].dropna()


# 基于物品的协同过滤推荐
def item_based_recommendation(user_id, top_k=4, n_neigh=78):
    # 检查用户是否在用户-物品矩阵中
    if user_id not in user_item_matrix.index:
        return pd.Series()

    # 获取用户对物品的评分
    user_ratings = get_user_ratings(user_id)
    # 初始化推荐列表
    recommendations = pd.Series(dtype=np.float64)

    for item, rating in user_ratings.items():
        # 检查物品是否在物品-用户矩阵的列中
        if item not in user_item_matrix.columns:
            continue

        # 获取当前物品与其他物品的相似度分数
        item_sim_scores = pd.Series(item_similarity[user_item_matrix.columns.get_loc(item)], index=user_item_matrix.columns).sort_values(ascending=False)
        # 选取前n_neigh个最相似的物品
        similar_items = item_sim_scores.index[:n_neigh]

        for idx, similar_item in enumerate(similar_items):
            # 跳过用户已经评分的物品
            if similar_item in user_ratings.index:
                continue
            # 根据相似物品的评分和相似度分数计算推荐得分
            recommendations.at[similar_item] = (recommendations.get(similar_item, 0) + item_sim_scores[similar_item] * rating)

    # 按照推荐得分对物品进行分组和求和
    recommendations = recommendations.groupby(recommendations.index).sum()
    # 返回得分最高的top_k个推荐物品
    return recommendations.sort_values(ascending=False).head(top_k)


# 示例推荐
print(item_based_recommendation(0))

190     1.208134
1438    0.984844
1538    0.980827
750     0.963872
dtype: float64


## 3. 基于交替最小二乘法（ALS）的推荐算法


In [9]:
import numpy as np
from scipy.sparse.linalg import svds

# 计算用户相似度和物品相似度矩阵
user_similarity = cosine_similarity(user_item_matrix.fillna(0))
item_similarity = cosine_similarity(user_item_matrix.fillna(0).T)


def compute_rmse(R, U, Vt):
    R_pred = np.dot(U, Vt)
    mse = np.sum((R - R_pred) ** 2) / np.count_nonzero(R)
    rmse = np.sqrt(mse)
    return rmse


def als_train(R, k=10, max_iter=10, tol=0.001):
    num_users, num_items = R.shape
    U = np.random.rand(num_users, k)
    Vt = np.random.rand(k, num_items)
    R_demeaned = R - np.mean(R, axis=1).reshape(-1, 1)

    for i in range(max_iter):
        # Fix Vt and solve for U
        for u in range(num_users):
            U[u, :] = np.linalg.solve(np.dot(Vt, Vt.T), np.dot(Vt, R_demeaned[u, :].T)).T

        # Fix U and solve for Vt
        for v in range(num_items):
            Vt[:, v] = np.linalg.solve(np.dot(U.T, U), np.dot(U.T, R_demeaned[:, v]))

        rmse = compute_rmse(R_demeaned, U, Vt)
        # print(f"Iteration {i+1}/{max_iter}, RMSE: {rmse}")

        if rmse < tol:
            break

    return U, Vt

# ALS矩阵分解
R = user_item_matrix.fillna(0).values
U, Vt = als_train(R, k=8, max_iter=20, tol=0.001)
predicted_ratings = np.dot(U, Vt) + np.mean(R, axis=1).reshape(-1, 1)
predicted_ratings_df = pd.DataFrame(
    predicted_ratings, columns=user_item_matrix.columns, index=user_item_matrix.index
)


# ALS推荐算法
def als_user_recommendation(user_id, top_k=4):
    if user_id not in predicted_ratings_df.index:
        return pd.Series()

    user_ratings = predicted_ratings_df.loc[user_id].sort_values(ascending=False)
    return user_ratings.head(top_k)


print(als_user_recommendation(0))

meal_code
57     0.499822
275    0.262748
86     0.246584
5      0.212530
Name: 0, dtype: float64


## 旧版

In [ ]:
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from scipy.sparse.linalg import svds

# 加载数据
train_ratings = pd.read_csv('train_ratings.csv')
validation_ratings = pd.read_csv('validation_ratings.csv')
test_ratings = pd.read_csv('test_ratings.csv')

# 创建用户-物品矩阵
user_item_matrix = train_ratings.pivot_table(
    index='user_code', columns='meal_code', values='Rating'
)

# 计算用户相似度和物品相似度矩阵
user_similarity = cosine_similarity(user_item_matrix.fillna(0))
item_similarity = cosine_similarity(user_item_matrix.fillna(0).T)

# ALS矩阵分解
R = user_item_matrix.fillna(0).values
user_ratings_mean = np.mean(R, axis=1)
R_demeaned = R - user_ratings_mean.reshape(-1, 1)
U, sigma, Vt = svds(R_demeaned, k=10)
sigma = np.diag(sigma)
predicted_ratings = np.dot(np.dot(U, sigma), Vt) + user_ratings_mean.reshape(-1, 1)
predicted_ratings_df = pd.DataFrame(
    predicted_ratings, columns=user_item_matrix.columns, index=user_item_matrix.index
)


# 获取某个用户的评分
def get_user_ratings(user_id):
    return user_item_matrix.loc[user_id].dropna()


# 基于用户的协同过滤推荐
def user_based_recommendation(user_id, top_k=4):
    if user_id not in user_item_matrix.index:
        return pd.Series()

    user_sim_scores = pd.Series(
        user_similarity[user_item_matrix.index.get_loc(user_id)], index=user_item_matrix.index
    )
    user_sim_scores = user_sim_scores.drop(user_id, errors='ignore').sort_values(ascending=False)
    similar_users = user_sim_scores.index[:top_k]

    recommendations = pd.Series(dtype=np.float64)
    for similar_user in similar_users:
        similar_user_ratings = get_user_ratings(similar_user)
        recommendations = pd.concat([recommendations, similar_user_ratings])

    recommendations = recommendations.groupby(recommendations.index).sum()
    user_rated_items = get_user_ratings(user_id).index
    recommendations = recommendations.drop(user_rated_items, errors='ignore')

    return recommendations.sort_values(ascending=False).head(top_k)


# 基于物品的协同过滤推荐
def item_based_recommendation(user_id, top_k=4):
    if user_id not in user_item_matrix.index:
        return pd.Series()

    user_ratings = get_user_ratings(user_id)
    recommendations = pd.Series(dtype=np.float64)

    for item, rating in user_ratings.items():
        if item not in user_item_matrix.columns:
            continue

        item_sim_scores = pd.Series(
            item_similarity[user_item_matrix.columns.get_loc(item)], index=user_item_matrix.columns
        )
        similar_items = item_sim_scores.sort_values(ascending=False).index[:top_k]

        for similar_item in similar_items:
            if similar_item in user_ratings.index:
                continue
            recommendations.at[similar_item] = (
                recommendations.get(similar_item, 0) + item_sim_scores[similar_item] * rating
            )

    recommendations = recommendations.groupby(recommendations.index).sum()
    return recommendations.sort_values(ascending=False).head(top_k)


# ALS推荐算法
def als_user_recommendation(user_id, top_k=4):
    if user_id not in predicted_ratings_df.index:
        return pd.Series()

    user_ratings = predicted_ratings_df.loc[user_id].sort_values(ascending=False)
    user_rated_items = get_user_ratings(user_id).index
    recommendations = user_ratings.drop(user_rated_items, errors='ignore')
    return recommendations.head(top_k)


# 计算评估参数
def calculate_evaluation_score(recommendation_func, test_data):
    hits = 0
    recall_total = 0
    precision_total = 0

    for user_id in test_data['user_code'].unique():
        user_test_ratings = test_data[test_data['user_code'] == user_id]
        recommended_items = recommendation_func(user_id).index.tolist()

        for item_id in user_test_ratings['meal_code']:
            if item_id in recommended_items:
                hits += 1
        recall_total += len(user_test_ratings['meal_code'])
        precision_total += len(recommended_items)

    precision_value = hits / precision_total if precision_total > 0 else 0
    recall_value = hits / recall_total if recall_total > 0 else 0

    return (
        precision_value,
        recall_value,
        2 * precision_value * recall_value / (precision_value + recall_value),
    )


# 计算测试集的准确率
print(
    "User-based CF Validation Evaluation:",
    calculate_evaluation_score(user_based_recommendation, validation_ratings),
)
print(
    "Item-based CF Validation Evaluation:",
    calculate_evaluation_score(item_based_recommendation, validation_ratings),
)
print(
    "ALS Validation Evaluation:",
    calculate_evaluation_score(als_user_recommendation, validation_ratings),
)

# 合并训练集和验证集
combined_train_validation_ratings = pd.concat([train_ratings, validation_ratings])

# 创建新的用户-物品矩阵
user_item_matrix = combined_train_validation_ratings.pivot_table(
    index='user_code', columns='meal_code', values='Rating'
)

# 重新计算用户相似度和物品相似度矩阵
user_similarity = cosine_similarity(user_item_matrix.fillna(0))
item_similarity = cosine_similarity(user_item_matrix.fillna(0).T)

# ALS矩阵分解
R = user_item_matrix.fillna(0).values
user_ratings_mean = np.mean(R, axis=1)
R_demeaned = R - user_ratings_mean.reshape(-1, 1)
U, sigma, Vt = svds(R_demeaned, k=10)
sigma = np.diag(sigma)
predicted_ratings = np.dot(np.dot(U, sigma), Vt) + user_ratings_mean.reshape(-1, 1)
predicted_ratings_df = pd.DataFrame(
    predicted_ratings, columns=user_item_matrix.columns, index=user_item_matrix.index
)

# 计算测试集的准确率
print(
    "User-based CF Test Evaluation:",
    calculate_evaluation_score(user_based_recommendation, test_ratings),
)
print(
    "Item-based CF Test Evaluation:",
    calculate_evaluation_score(item_based_recommendation, test_ratings),
)
print("ALS Test Evaluation:", calculate_evaluation_score(als_user_recommendation, test_ratings))

## 最终版

In [12]:
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics import precision_recall_fscore_support
from scipy.sparse.linalg import svds

# 加载数据
train_ratings = pd.read_csv('train_ratings.csv')
validation_ratings = pd.read_csv('validation_ratings.csv')
test_ratings = pd.read_csv('test_ratings.csv')

# 创建用户-物品矩阵
user_item_matrix = train_ratings.pivot_table(
    index='user_code', columns='meal_code', values='Rating'
)

# 计算用户相似度和物品相似度矩阵
user_similarity = cosine_similarity(user_item_matrix.fillna(0))
item_similarity = cosine_similarity(user_item_matrix.fillna(0).T)


def compute_rmse(R, U, Vt):
    R_pred = np.dot(U, Vt)
    mse = np.sum((R - R_pred) ** 2) / np.count_nonzero(R)
    rmse = np.sqrt(mse)
    return rmse


def als_train(R, k=10, max_iter=10, tol=0.001):
    num_users, num_items = R.shape
    U = np.random.rand(num_users, k)
    Vt = np.random.rand(k, num_items)
    R_demeaned = R - np.mean(R, axis=1).reshape(-1, 1)

    for i in range(max_iter):
        # Fix Vt and solve for U
        for u in range(num_users):
            U[u, :] = np.linalg.solve(np.dot(Vt, Vt.T), np.dot(Vt, R_demeaned[u, :].T)).T

        # Fix U and solve for Vt
        for v in range(num_items):
            Vt[:, v] = np.linalg.solve(np.dot(U.T, U), np.dot(U.T, R_demeaned[:, v]))

        rmse = compute_rmse(R_demeaned, U, Vt)
        # print(f"Iteration {i+1}/{max_iter}, RMSE: {rmse}")

        if rmse < tol:
            break

    return U, Vt


# ALS矩阵分解
R = user_item_matrix.fillna(0).values
U, Vt = als_train(R, k=8, max_iter=20, tol=0.001)
predicted_ratings = np.dot(U, Vt) + np.mean(R, axis=1).reshape(-1, 1)
predicted_ratings_df = pd.DataFrame(
    predicted_ratings, columns=user_item_matrix.columns, index=user_item_matrix.index
)


# 获取某个用户的评分
def get_user_ratings(user_id):
    return user_item_matrix.loc[user_id].dropna()


# 基于用户的协同过滤推荐
def user_based_recommendation(user_id, top_k=2, n_neigh=74):
    if user_id not in user_item_matrix.index:
        return pd.Series()

    user_sim_scores = pd.Series(
        user_similarity[user_item_matrix.index.get_loc(user_id)], index=user_item_matrix.index
    )
    user_sim_scores = user_sim_scores.drop(user_id, errors='ignore').sort_values(ascending=False)
    similar_users = user_sim_scores.index[:n_neigh]

    recommendations = pd.Series(dtype=np.float64)
    for idx, similar_user in enumerate(similar_users):
        similar_user_ratings = get_user_ratings(similar_user)
        recommendations = pd.concat(
            [recommendations, similar_user_ratings * user_sim_scores.iloc[idx]]
        )

    recommendations = recommendations.groupby(recommendations.index).sum()
    user_rated_items = get_user_ratings(user_id).index
    recommendations = recommendations.drop(user_rated_items, errors='ignore')

    return recommendations.sort_values(ascending=False).head(top_k)


# 基于物品的协同过滤推荐
def item_based_recommendation(user_id, top_k=2, n_neigh=67):
    if user_id not in user_item_matrix.index:
        return pd.Series()

    user_ratings = get_user_ratings(user_id)
    recommendations = pd.Series(dtype=np.float64)

    for item, rating in user_ratings.items():
        if item not in user_item_matrix.columns:
            continue

        item_sim_scores = pd.Series(
            item_similarity[user_item_matrix.columns.get_loc(item)], index=user_item_matrix.columns
        ).sort_values(ascending=False)
        similar_items = item_sim_scores.index[:n_neigh]

        for idx, similar_item in enumerate(similar_items):
            if similar_item in user_ratings.index:
                continue
            recommendations.at[similar_item] = (
                recommendations.get(similar_item, 0)
                + item_sim_scores[similar_item] * rating
            )

    recommendations = recommendations.groupby(recommendations.index).sum()
    return recommendations.sort_values(ascending=False).head(top_k)


# ALS推荐算法
def als_user_recommendation(user_id, top_k=5):
    if user_id not in predicted_ratings_df.index:
        return pd.Series()

    user_ratings = predicted_ratings_df.loc[user_id].sort_values(ascending=False)
    return user_ratings.head(top_k)


# 计算评估参数
def calculate_evaluation_score(recommendation_func, test_data):
    hits = 0
    recall_total = 0
    precision_total = 0

    for user_id in test_data['user_code'].unique():
        user_test_ratings = test_data[test_data['user_code'] == user_id]
        recommended_items = recommendation_func(user_id).index.tolist()

        for item_id in user_test_ratings['meal_code']:
            if item_id in recommended_items:
                hits += 1
        recall_total += len(user_test_ratings['meal_code'])
        precision_total += len(recommended_items)

    precision_value = hits / precision_total if precision_total > 0 else 0
    recall_value = hits / recall_total if recall_total > 0 else 0

    return (
        precision_value,
        recall_value,
        2 * precision_value * recall_value / (precision_value + recall_value),
    )

# 计算测试集的准确率
print(
    "User-based CF Validation Evaluation:",
    calculate_evaluation_score(user_based_recommendation, validation_ratings),
)
print(
    "Item-based CF Validation Evaluation:",
    calculate_evaluation_score(item_based_recommendation, validation_ratings),
)
# 计算验证集和测试集的F1分数
print(
    "ALS Validation Evaluation:",
    calculate_evaluation_score(als_user_recommendation, validation_ratings),
)

# 合并训练集和验证集
combined_train_validation_ratings = pd.concat([train_ratings, validation_ratings])

# 创建新的用户-物品矩阵
user_item_matrix = combined_train_validation_ratings.pivot_table(
    index='user_code', columns='meal_code', values='Rating'
)

# 重新计算用户相似度和物品相似度矩阵
user_similarity = cosine_similarity(user_item_matrix.fillna(0))
item_similarity = cosine_similarity(user_item_matrix.fillna(0).T)

# ALS矩阵分解
R = user_item_matrix.fillna(0).values
U, Vt = als_train(R, k=8, max_iter=20, tol=0.001)
predicted_ratings = np.dot(U, Vt) + np.mean(R, axis=1).reshape(-1, 1)
predicted_ratings_df = pd.DataFrame(
    predicted_ratings, columns=user_item_matrix.columns, index=user_item_matrix.index
)

# 计算测试集的准确率
print(
    "User-based CF Test Evaluation:",
    calculate_evaluation_score(user_based_recommendation, test_ratings),
)
print(
    "Item-based CF Test Evaluation:",
    calculate_evaluation_score(item_based_recommendation, test_ratings),
)
print("ALS Test Evaluation:", calculate_evaluation_score(als_user_recommendation, test_ratings))

User-based CF Validation Evaluation: (0.087109375, 0.12015086206896551, 0.10099637681159421)
Item-based CF Validation Evaluation: (0.083203125, 0.11476293103448276, 0.09646739130434784)
ALS Validation Evaluation: (0.02828125, 0.09752155172413793, 0.0438468992248062)
User-based CF Test Evaluation: (0.09698865858427845, 0.13358470239698358, 0.11238246289792683)
Item-based CF Test Evaluation: (0.10500586624951115, 0.14462698626447618, 0.121672142290699)
ALS Test Evaluation: (0.03269456394211967, 0.11257743064907083, 0.05067280882531216)


## UI

In [ ]:
import sys
from PyQt5.QtWidgets import QApplication, QMainWindow, QVBoxLayout, QHBoxLayout, QComboBox, QPushButton, QListWidget, QLabel, QWidget, QGroupBox
from PyQt5.QtCore import Qt

class RecommendationApp(QMainWindow):
    def __init__(self):
        super().__init__()
        self.setWindowTitle("推荐系统")
        self.setGeometry(100, 100, 800, 600)
        self.initUI()

    def initUI(self):
        # 创建主窗口部件
        central_widget = QWidget()
        self.setCentralWidget(central_widget)
        
        # 创建主布局
        main_layout = QVBoxLayout(central_widget)
        
        # 顶部用户选择部分
        user_selection_layout = QHBoxLayout()
        self.user_combo = QComboBox()
        self.user_combo.setEditable(True)
        self.user_combo.addItems(["User1", "User2", "User3"])  # 示例用户ID
        self.user_button = QPushButton("确定")
        self.user_button.clicked.connect(self.load_user_data)
        
        user_selection_layout.addWidget(QLabel("选择用户:"))
        user_selection_layout.addWidget(self.user_combo)
        user_selection_layout.addWidget(self.user_button)
        
        main_layout.addLayout(user_selection_layout, 1)
        
        # 中间部分布局
        middle_layout = QHBoxLayout()
        
        # 左侧已选菜品部分
        selected_meals_group = QGroupBox("已选菜品")
        selected_meals_layout = QVBoxLayout(selected_meals_group)
        
        self.meal_combo = QComboBox()
        self.meal_combo.addItems(["Meal1", "Meal2", "Meal3"])  # 示例菜品名称
        self.meal_button = QPushButton("添加")
        self.meal_button.clicked.connect(self.add_meal)
        self.selected_meals_list = QListWidget()
        
        meal_selection_layout = QHBoxLayout()
        meal_selection_layout.addWidget(self.meal_combo)
        meal_selection_layout.addWidget(self.meal_button)
        
        selected_meals_layout.addLayout(meal_selection_layout)
        selected_meals_layout.addWidget(self.selected_meals_list)
        
        middle_layout.addWidget(selected_meals_group, 2)
        
        # 右侧推荐部分
        recommendation_layout = QVBoxLayout()
        
        self.user_based_group = self.create_recommendation_group("基于用户的协同过滤")
        self.item_based_group = self.create_recommendation_group("基于物品的协同过滤")
        self.als_group = self.create_recommendation_group("ALS推荐算法")
        
        recommendation_layout.addWidget(self.user_based_group)
        recommendation_layout.addWidget(self.item_based_group)
        recommendation_layout.addWidget(self.als_group)
        
        middle_layout.addLayout(recommendation_layout, 6)
        
        main_layout.addLayout(middle_layout, 9)

    def create_recommendation_group(self, title):
        group = QGroupBox(title)
        layout = QVBoxLayout(group)
        
        for i in range(4):
            btn = QPushButton(f"推荐菜品 {i+1}")
            layout.addWidget(btn)
        
        return group

    def load_user_data(self):
        user_id = self.user_combo.currentText()
        # 这里可以添加加载用户数据的逻辑
        print(f"加载用户数据: {user_id}")

    def add_meal(self):
        meal = self.meal_combo.currentText()
        self.selected_meals_list.addItem(meal)
        # 这里可以添加用户选择菜品的逻辑
        print(f"添加菜品: {meal}")

if __name__ == "__main__":
    app = QApplication(sys.argv)
    window = RecommendationApp()
    window.show()
    sys.exit(app.exec_())


## 计算最佳Top_K

User-based CF Validation Evaluation: (0.03390856796851347, 0.09051724137931035, 0.04933558475882827)

Item-based CF Validation Evaluation: (0.06826716738197425, 0.13712284482758622, 0.09115329512893984)

ALS Validation Evaluation: (0.0284375, 0.0980603448275862, 0.04408914728682171)

User-based CF Test Evaluation: (0.04339132173565287, 0.11688661459736062, 0.06328837039737513)

Item-based CF Test Evaluation: (0.07759081196581197, 0.1564772421222731, 0.10374073743415767)

ALS Test Evaluation: (0.032850997262416894, 0.11311607864260706, 0.05091526245605528)

In [ ]:
# 基于用户的协同过滤推荐
def user_based_recommendation(user_id, top_k=4,n_neigh=4):
    if user_id not in user_item_matrix.index:
        return pd.Series()

    user_sim_scores = pd.Series(
        user_similarity[user_item_matrix.index.get_loc(user_id)], index=user_item_matrix.index
    )
    user_sim_scores = user_sim_scores.drop(user_id, errors='ignore').sort_values(ascending=False)
    similar_users = user_sim_scores.index[:n_neigh]

    recommendations = pd.Series(dtype=np.float64)
    for idx, similar_user in enumerate(similar_users):
        similar_user_ratings = get_user_ratings(similar_user)
        recommendations = pd.concat(
            [recommendations, similar_user_ratings * user_sim_scores.iloc[idx]]
        )

    recommendations = recommendations.groupby(recommendations.index).sum()
    user_rated_items = get_user_ratings(user_id).index
    recommendations = recommendations.drop(user_rated_items, errors='ignore')

    return recommendations.sort_values(ascending=False).head(top_k)

# 合并训练集和验证集
combined_train_validation_ratings = pd.concat([train_ratings, validation_ratings])

# 创建新的用户-物品矩阵
user_item_matrix = combined_train_validation_ratings.pivot_table(
    index='user_code', columns='meal_code', values='Rating'
)

# 重新计算用户相似度和物品相似度矩阵
user_similarity = cosine_similarity(user_item_matrix.fillna(0))
item_similarity = cosine_similarity(user_item_matrix.fillna(0).T)

for i in range(25,30):
    # 计算测试集的准确率

    hits = 0
    recall_total = 0
    precision_total = 0

    for user_id in test_ratings['user_code'].unique():
        user_test_ratings = test_ratings[test_ratings['user_code'] == user_id]
        recommended_items = user_based_recommendation(user_id, i).index.tolist()

        for item_id in user_test_ratings['meal_code']:
            if item_id in recommended_items:
                hits += 1
        recall_total += len(user_test_ratings['meal_code'])
        precision_total += len(recommended_items)

    precision_value = hits / precision_total if precision_total > 0 else 0
    recall_value = hits / recall_total if recall_total > 0 else 0

    print(
        i,
        "Item-based CF Test Evaluation:",
        precision_value,
        recall_value,
        2 * precision_value * recall_value / (precision_value + recall_value),
    )

In [4]:
import numpy as np
import pandas as pd
from hyperopt import fmin, tpe, hp, Trials
from sklearn.metrics.pairwise import cosine_similarity


# 基于物品的协同过滤推荐
def item_based_recommendation(user_id, top_k=3, n_neigh=3):
    if user_id not in user_item_matrix.index:
        return pd.Series()

    user_ratings = get_user_ratings(user_id)
    recommendations = pd.Series(dtype=np.float64)

    for item, rating in user_ratings.items():
        if item not in user_item_matrix.columns:
            continue

        item_sim_scores = pd.Series(
            item_similarity[user_item_matrix.columns.get_loc(item)], index=user_item_matrix.columns
        ).sort_values(ascending=False)
        similar_items = item_sim_scores.index[:n_neigh]

        for idx, similar_item in enumerate(similar_items):
            if similar_item in user_ratings.index:
                continue
            recommendations.at[similar_item] = (
                recommendations.get(similar_item, 0) + item_sim_scores[similar_item] * rating
            )

    recommendations = recommendations.groupby(recommendations.index).sum()
    return recommendations.sort_values(ascending=False).head(top_k)


# 合并训练集和验证集
combined_train_validation_ratings = pd.concat([train_ratings, validation_ratings])

# 创建新的用户-物品矩阵
user_item_matrix = combined_train_validation_ratings.pivot_table(
    index='user_code', columns='meal_code', values='Rating'
)

# 重新计算用户相似度和物品相似度矩阵
user_similarity = cosine_similarity(user_item_matrix.fillna(0))
item_similarity = cosine_similarity(user_item_matrix.fillna(0).T)


# 定义目标函数
def objective(params):
    top_k = int(params['top_k'])
    n_neigh = int(params['n_neigh'])

    hits = 0
    recall_total = 0
    precision_total = 0

    for user_id in test_ratings['user_code'].unique():
        user_test_ratings = test_ratings[test_ratings['user_code'] == user_id]
        recommended_items = item_based_recommendation(user_id, top_k, n_neigh).index.tolist()

        for item_id in user_test_ratings['meal_code']:
            if item_id in recommended_items:
                hits += 1
        recall_total += len(user_test_ratings['meal_code'])
        precision_total += len(recommended_items)

    precision_value = hits / precision_total if precision_total > 0 else 0
    recall_value = hits / recall_total if recall_total > 0 else 0

    if precision_value + recall_value > 0:
        f1_value = 2 * precision_value * recall_value / (precision_value + recall_value)
    else:
        f1_value = 0

    print(top_k, n_neigh, (precision_value, recall_value, f1_value))

    return -f1_value  # Hyperopt最小化目标函数，所以我们返回负的F1值


# 定义搜索空间
space = {'top_k': hp.quniform('top_k', 1, 5, 1), 'n_neigh': hp.quniform('n_neigh', 1, 160, 1)}

# 优化
trials = Trials()
best = fmin(fn=objective, space=space, algo=tpe.suggest, max_evals=100, trials=trials)

print("Best hyperparameters:", best)

[I 2024-07-14 22:14:08,033] A new study created in memory with name: no-name-9d5796eb-044d-4679-b1a7-fa2a10a179c2
[I 2024-07-14 22:30:00,627] Trial 5 finished with value: -0.1084529505582137 and parameters: {'top_k': 1, 'n_neigh': 12}. Best is trial 5 with value: -0.1084529505582137.


1 12 (0.13296832225263983, 0.09157015890115809, 0.1084529505582137)


[I 2024-07-14 22:30:48,693] Trial 7 finished with value: -0.10759629868732515 and parameters: {'top_k': 4, 'n_neigh': 19}. Best is trial 5 with value: -0.1084529505582137.


4 19 (0.07332811888932343, 0.20199299757608402, 0.10759629868732515)


[I 2024-07-14 22:32:34,272] Trial 6 finished with value: -0.11700632466619817 and parameters: {'top_k': 3, 'n_neigh': 24}. Best is trial 6 with value: -0.11700632466619817.


...

4 35 (0.07606570199452484, 0.20953406948559117, 0.11161322717165195)


[I 2024-07-14 22:38:20,304] Trial 2 finished with value: -0.10972886762360447 and parameters: {'top_k': 1, 'n_neigh': 51}. Best is trial 6 with value: -0.11700632466619817.


1 51 (0.13453265545561205, 0.09264745488823053, 0.10972886762360447)


[I 2024-07-14 22:50:55,922] Trial 4 finished with value: -0.12076583210603829 and parameters: {'top_k': 2, 'n_neigh': 97}. Best is trial 4 with value: -0.12076583210603829.


2 97 (0.10422369964802503, 0.14354969027740372, 0.12076583210603829)


## 可视化

In [2]:
import json
import re
import sys
import threading

import numpy as np
import pandas as pd
from PyQt5.QtCore import Qt, QTimer
from PyQt5.QtWidgets import (
    QApplication,
    QComboBox,
    QGroupBox,
    QHBoxLayout,
    QLabel,
    QListWidget,
    QMainWindow,
    QPushButton,
    QVBoxLayout,
    QWidget,
)
from sklearn.metrics.pairwise import cosine_similarity


class RecommendationApp(QMainWindow):
    def __init__(self):
        super().__init__()
        self.setWindowTitle("推荐系统")
        self.setGeometry(100, 100, 800, 600)

        # 加载数据
        self.load_data()

        # 初始化UI
        self.initUI()

        # 设置推荐变量
        self.recommendation_started = False

    def load_data(self):
        # 读取 SQL 文件
        with open('./data/meal_list.sql', 'r', encoding='utf-8') as f:
            sql_content = f.read()

        # 提取插入语句中的数据
        insert_statements = re.findall(
            r"INSERT INTO `meal_list` VALUES \((.*?)\);", sql_content, re.S
        )

        # 解析数据
        meal_data = []
        for statement in insert_statements:
            rows = statement.split("),(")
            for row in rows:
                row_data = row.split(',')
                meal_data.append([int(row_data[0]), row_data[1].strip("'"), row_data[2].strip("'")])

        # 创建 DataFrame
        self.meal_df = pd.DataFrame(meal_data, columns=['meal_no', 'MealID', 'meal_name'])

        # 读取 JSON 文件
        with open('./data/MealRatings_201705_201706.json', 'r', encoding='utf-8') as f:
            ratings_data = json.load(f)

        # 创建 DataFrame
        self.ratings_df = pd.DataFrame(ratings_data)

        # 处理重复评分记录
        self.ratings_df.sort_values(
            by=['UserID', 'MealID', 'ReviewTime'], ascending=[True, True, False], inplace=True
        )
        self.ratings_df.drop_duplicates(subset=['UserID', 'MealID'], keep='first', inplace=True)

        # 编码用户和菜品
        self.user_id_map = {
            user_id: idx for idx, user_id in enumerate(self.ratings_df['UserID'].unique())
        }
        self.meal_id_map = {
            meal_id: idx for idx, meal_id in enumerate(self.ratings_df['MealID'].unique())
        }

        self.ratings_df['user_code'] = self.ratings_df['UserID'].map(self.user_id_map)
        self.ratings_df['meal_code'] = self.ratings_df['MealID'].map(self.meal_id_map)

    def initUI(self):
        # 创建主窗口部件
        central_widget = QWidget()
        self.setCentralWidget(central_widget)

        # 创建主布局
        main_layout = QVBoxLayout(central_widget)

        # 顶部用户选择部分
        user_selection_layout = QHBoxLayout()
        self.user_combo = QComboBox()
        self.user_combo.setEditable(True)
        self.user_combo.addItems(self.ratings_df['UserID'].unique().tolist())
        self.user_button = QPushButton("确定")
        self.user_button.clicked.connect(self.load_user_data)

        user_selection_layout.addWidget(QLabel("选择用户:"))
        user_selection_layout.addWidget(self.user_combo)
        user_selection_layout.addWidget(self.user_button)

        main_layout.addLayout(user_selection_layout, 1)

        # 中间部分布局
        middle_layout = QHBoxLayout()

        # 左侧已选菜品部分
        selected_meals_group = QGroupBox("已选菜品")
        selected_meals_layout = QVBoxLayout(selected_meals_group)

        self.meal_combo = QComboBox()
        self.meal_combo.addItems(self.meal_df['meal_name'].tolist())
        self.meal_button = QPushButton("添加")
        self.meal_button.clicked.connect(self.add_meal)
        self.selected_meals_list = QListWidget()
        self.selected_meals_list.itemDoubleClicked.connect(self.remove_meal)

        meal_selection_layout = QHBoxLayout()
        meal_selection_layout.addWidget(self.meal_combo)
        meal_selection_layout.addWidget(self.meal_button)

        selected_meals_layout.addLayout(meal_selection_layout)
        selected_meals_layout.addWidget(self.selected_meals_list)

        middle_layout.addWidget(selected_meals_group, 2)

        # 右侧推荐部分
        recommendation_layout = QVBoxLayout()

        self.recommend_button = QPushButton("开始推荐")
        self.recommend_button.clicked.connect(self.start_recommendation)
        recommendation_layout.addWidget(self.recommend_button)

        self.user_based_group = self.create_recommendation_group("基于用户的协同过滤")
        self.item_based_group = self.create_recommendation_group("基于物品的协同过滤")
        self.als_group = self.create_recommendation_group("ALS推荐算法")

        recommendation_layout.addWidget(self.user_based_group)
        recommendation_layout.addWidget(self.item_based_group)
        recommendation_layout.addWidget(self.als_group)

        middle_layout.addLayout(recommendation_layout, 6)

        main_layout.addLayout(middle_layout, 9)

        # 初始化已选菜品集合
        self.selected_meals_set = set()

    def create_recommendation_group(self, title):
        group = QGroupBox(title)
        layout = QVBoxLayout(group)

        for i in range(4):
            btn = QPushButton(f"推荐菜品 {i+1}")
            btn.clicked.connect(lambda checked, b=btn: self.order_meal(b.text()))
            layout.addWidget(btn)

        return group

    def order_meal(self, meal):
        if meal not in self.selected_meals_set:
            self.selected_meals_set.add(meal)
            self.selected_meals_list.addItem(meal)
            print(f"添加菜品: {meal}")
            self.check_recommendation_status()

    def load_user_data(self):
        user_id = self.user_combo.currentText()
        if user_id in self.user_id_map:
            user_code = self.user_id_map[user_id]
            user_meals = self.ratings_df[self.ratings_df['user_code'] == user_code]['MealID']
            meal_names = self.meal_df.set_index('MealID').loc[user_meals]['meal_name'].tolist()

            self.selected_meals_list.clear()
            self.selected_meals_set = set(meal_names)
            self.selected_meals_list.addItems(self.selected_meals_set)

            print(f"加载用户数据: {user_id}")
        else:
            self.selected_meals_list.clear()
            self.selected_meals_set.clear()
            print(f"新用户: {user_id}")

    def add_meal(self):
        meal = self.meal_combo.currentText()
        if meal not in self.selected_meals_set:
            self.selected_meals_set.add(meal)
            self.selected_meals_list.addItem(meal)
            print(f"添加菜品: {meal}")
            self.check_recommendation_status()

    def remove_meal(self, item):
        meal = item.text()
        if meal in self.selected_meals_set:
            self.selected_meals_set.remove(meal)
            self.selected_meals_list.takeItem(self.selected_meals_list.row(item))
            print(f"删除菜品: {meal}")
            self.check_recommendation_status()

    def check_recommendation_status(self):
        print("CHANGE")
        if self.recommendation_started:
            # 启动推荐线程
            threading.Thread(target=self.run_recommendations).start()

    def start_recommendation(self):

        # 更新推荐状态
        self.recommendation_started = True
        self.recommend_button.setEnabled(False)

        # 启动推荐线程
        threading.Thread(target=self.run_recommendations).start()

    def run_recommendations(self):

        # 获取当前用户所选择的菜品记录
        selected_meals = list(self.selected_meals_set)
        new_user_code = len(self.user_id_map)  # 为新用户创建新的 user_code
        self.ratings_df_new = self.ratings_df.copy()

        # 为新的 user_code 添加评分记录，假设评分为 5
        for meal in selected_meals:
            meal_id = self.meal_df[self.meal_df['meal_name'] == meal]['MealID'].values[0]
            meal_code = self.meal_id_map[meal_id]
            self.ratings_df_new = self.ratings_df_new._append(
                {
                    'UserID': f'new_user_{new_user_code}',
                    'MealID': meal_id,
                    'Rating': 5,
                    'ReviewTime': pd.Timestamp.now(),
                    'user_code': new_user_code,
                    'meal_code': meal_code,
                },
                ignore_index=True,
            )
        # 创建用户-物品矩阵
        user_item_matrix = self.ratings_df_new.pivot_table(
            index='user_code', columns='meal_code', values='Rating'
        )

        # 计算用户相似度和物品相似度矩阵
        user_similarity = cosine_similarity(user_item_matrix.fillna(0))
        item_similarity = cosine_similarity(user_item_matrix.fillna(0).T)
        print(user_item_matrix.iloc[-1].max())

        def compute_rmse(R, U, Vt):
            R_pred = np.dot(U, Vt)
            mse = np.sum((R - R_pred) ** 2) / np.count_nonzero(R)
            rmse = np.sqrt(mse)
            return rmse

        def als_train(R, k=10, max_iter=10, tol=0.001):
            num_users, num_items = R.shape
            U = np.random.rand(num_users, k)
            Vt = np.random.rand(k, num_items)
            R_demeaned = R - np.mean(R, axis=1).reshape(-1, 1)

            for i in range(max_iter):
                # Fix Vt and solve for U
                for u in range(num_users):
                    U[u, :] = np.linalg.solve(np.dot(Vt, Vt.T), np.dot(Vt, R_demeaned[u, :].T)).T

                # Fix U and solve for Vt
                for v in range(num_items):
                    Vt[:, v] = np.linalg.solve(np.dot(U.T, U), np.dot(U.T, R_demeaned[:, v]))

                rmse = compute_rmse(R_demeaned, U, Vt)
                # print(f"Iteration {i+1}/{max_iter}, RMSE: {rmse}")

                if rmse < tol:
                    break

            return U, Vt

        # ALS矩阵分解
        R = user_item_matrix.fillna(0).values
        U, Vt = als_train(R, k=8, max_iter=4, tol=0.001)
        predicted_ratings = np.dot(U, Vt) + np.mean(R, axis=1).reshape(-1, 1)
        predicted_ratings_df = pd.DataFrame(
            predicted_ratings, columns=user_item_matrix.columns, index=user_item_matrix.index
        )

        # 获取某个用户的评分
        def get_user_ratings(user_id):
            return user_item_matrix.loc[user_id].dropna()

        # 基于用户的协同过滤推荐
        def user_based_recommendation(user_id, top_k=4):
            if user_id not in user_item_matrix.index:
                return pd.Series()

            user_sim_scores = pd.Series(
                user_similarity[user_item_matrix.index.get_loc(user_id)],
                index=user_item_matrix.index,
            )
            user_sim_scores = user_sim_scores.drop(user_id, errors='ignore').sort_values(
                ascending=False
            )
            similar_users = user_sim_scores.index[: 4 * top_k]

            recommendations = pd.Series(dtype=np.float64)
            for similar_user in similar_users:
                similar_user_ratings = get_user_ratings(similar_user)
                recommendations = pd.concat([recommendations, similar_user_ratings])

            recommendations = recommendations.groupby(recommendations.index).sum()
            user_rated_items = get_user_ratings(user_id).index
            recommendations = recommendations.drop(user_rated_items, errors='ignore')

            return recommendations.sort_values(ascending=False).head(top_k)

        # 基于物品的协同过滤推荐
        def item_based_recommendation(user_id, top_k=3):
            if user_id not in user_item_matrix.index:
                return pd.Series()

            user_ratings = get_user_ratings(user_id)
            recommendations = pd.Series(dtype=np.float64)

            for item, rating in user_ratings.items():
                if item not in user_item_matrix.columns:
                    continue

                item_sim_scores = pd.Series(
                    item_similarity[user_item_matrix.columns.get_loc(item)],
                    index=user_item_matrix.columns,
                )
                similar_items = item_sim_scores.sort_values(ascending=False).index[: 4 * top_k]

                for similar_item in similar_items:
                    if similar_item in user_ratings.index:
                        continue
                    recommendations.at[similar_item] = (
                        recommendations.get(similar_item, 0)
                        + item_sim_scores[similar_item] * rating
                    )

            recommendations = recommendations.groupby(recommendations.index).sum()
            return recommendations.sort_values(ascending=False).head(top_k)

        # ALS推荐算法
        def als_user_recommendation(user_id, top_k=4):
            if user_id not in predicted_ratings_df.index:
                return pd.Series()

            user_ratings = predicted_ratings_df.loc[user_id].sort_values(ascending=False)
            user_rated_items = get_user_ratings(user_id).index
            recommendations = user_ratings.drop(user_rated_items, errors='ignore')
            return recommendations.head(top_k)

        # 三个推荐算法并行运行
        user_based_result = user_based_recommendation(new_user_code, top_k=4)
        item_based_result = item_based_recommendation(new_user_code, top_k=4)
        als_result = als_user_recommendation(new_user_code, top_k=4)
        print(user_based_result, item_based_result, als_result)

        # 更新UI
        self.update_recommendation_group(self.user_based_group, user_based_result)
        self.update_recommendation_group(self.item_based_group, item_based_result)
        self.update_recommendation_group(self.als_group, als_result)

        # 定义每个推荐算法的函数
        def run_user_based_recommendation():
            user_based_result = user_based_recommendation(new_user_code, top_k=4)
            self.update_recommendation_group(self.user_based_group, user_based_result)

        def run_item_based_recommendation():
            item_based_result = item_based_recommendation(new_user_code, top_k=4)
            self.update_recommendation_group(self.item_based_group, item_based_result)

        def run_als_recommendation():
            als_result = als_user_recommendation(new_user_code, top_k=4)
            self.update_recommendation_group(self.als_group, als_result)

        # 创建线程来并行运行推荐算法
        threads = []

        for func in [
            run_user_based_recommendation,
            run_item_based_recommendation,
            run_als_recommendation,
        ]:
            thread = threading.Thread(target=lambda f: threads.append(f()), args=(func,))
            thread.start()

        for thread in threads:
            thread.join()

    def update_recommendation_group(self, group, recommendations):
        for i, meal_code in enumerate(recommendations.index):
            meal_name = self.meal_df[
                self.meal_df['MealID'] == list(self.meal_id_map.keys())[meal_code]
            ]['meal_name'].values[0]
            group.layout().itemAt(i).widget().setText(meal_name)


if __name__ == "__main__":
    app = QApplication(sys.argv)
    window = RecommendationApp()
    window.show()
    sys.exit(app.exec_())

加载用户数据: A0705654XT5UCAYOY7TH
5.0
57     24.0
283    15.0
212    14.0
274    13.0
dtype: float64 1538    2.742636
599     2.102806
588     1.933729
283     1.924074
dtype: float64 meal_code
275    1.070987
57     1.059990
212    0.823227
121    0.777558
Name: 5130, dtype: float64
添加菜品: 冬瓜茶
CHANGE
5.0
212    24.0
190    18.0
220    12.0
283    10.0
dtype: float64 1538    2.742636
190     2.390222
599     2.102806
588     1.933729
dtype: float64 meal_code
212    1.583818
78     1.129002
85     1.076505
64     0.783548
Name: 5130, dtype: float64
添加菜品: 家常红烧鸡块
CHANGE
5.0
212    14.0
750     8.0
448     8.0
342     5.0
dtype: float64 1538    2.742636
599     2.102806
588     1.933729
283     1.924074
dtype: float64 meal_code
86     1.898079
78     1.447399
275    1.017308
85     0.955703
Name: 5130, dtype: float64
添加菜品: 芝士焗明虾
CHANGE
5.0
212    24.0
372    13.0
119    10.0
283    10.0
dtype: float64 1538    2.742636
283     2.438703
389     2.197492
599     2.102806
dtype: float64 meal_code
29

SystemExit: 0

d:\ProgramData\Anaconda3\envs\ML\lib\site-packages\IPython\core\interactiveshell.py:3534: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
